<a href="https://colab.research.google.com/github/amandatz/linear-programming/blob/main/OptModAula2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
using Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed CodecBzip2 ───────── v0.8.5
   Installed StructTypes ──────── v1.11.0
   Installed JSON3 ────────────── v1.14.3
   Installed BenchmarkTools ───── v1.6.3
   Installed MutableArithmetics ─ v1.6.7
   Installed MathOptInterface ─── v1.46.0
   Installed JuMP ─────────────── v1.29.3
    Updating `~/.julia/environments/v1.11/Project.toml`
  [4076af6c] + JuMP v1.29.3
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.3
  [523fee87] + CodecBzip2 v0.8.5
  [0f8b85d8] + JSON3 v1.14.3
  [4076af6c] + JuMP v1.29.3
  [b8f27783] + MathOptInterface v1.46.0
  [d8a4904e] + MutableArithmetics v1.6.7
  [856f2bd8] + StructTypes v1.11.0
  [9abbd945] + Profile v1.11.0
Precompiling project...
   7189.1 ms  ✓ CodecBzip2
   8974.5 ms  ✓ StructTypes
   4637.4 ms  ✓ BenchmarkTools
  21756.6 ms  ✓ MutableArithmetics
  24696.7 ms  ✓ JSON3
  97060.3 ms  ✓ MathOptInterface
 

# Pastesian

A Pastesian é uma fábrica de massas familiar que está planejando a produção de lasanhas para os próximos 4 meses.

As lasanhas são preparadas e imediatamente congeladas para distribuição. Eles têm alguns freezers na fábrica onde podem armazenar o estoque por várias semanas antes de enviá-las para o mercado.

Com base nas vendas de anos anteriores, a Pastesian espera uma demanda de 200, 350, 150 e 250 lasanhas para os meses 1, 2, 3 e 4, respectivamente, a qual deve ser atendida exatamente. Atualmente, a Pastesian tem apenas 50 lasanhas em estoque.

Um fator complicador é que o custo da mão de obra aumentará nos próximos meses, pois a fábrica está localizada em uma cidade turística com muitas oportunidades de emprego temporário durante a alta temporada. Isso se traduz em um custo de produção variável de R\$5,50, R$7,20, R\$8,80 e R\$10,90 por lasanha para os meses 1, 2, 3 e 4, respectivamente.

Além disso, os custos de eletricidade também variam ao longo da temporada. Como resultado, o custo para manter uma lasanha em estoque do mês 1 para o 2 é de $1,30, do mês 2 para o 3 é de R\$1,95 e do mês 3 para o 4 é de R\$2,20.

Como a Pastesian deve planejar suas operações para os próximos meses, assumindo que terminará a temporada sem lasanhas em estoque?

## Parâmetros

In [2]:
# Períodos (mês).
T = [1, 2, 3, 4]
# Demanda (unidade) em cada mês.
d = [200, 350, 150, 250]
# Lasanhas (unidade) em estoque inicialmente.
ei = 50
# Custo (R$/unidade) de produção em cada mês.
cp = [5.5, 7.2, 8.8, 10.9]
# Custo (R$/unidade) de manter as lasanhas em estoque, de um mês para o seguinte.
ce = [1.3, 1.95, 2.2];

Pergunta 1: Por que não temos o custo de se manter as 50 inicias  lasanhas em estoque?

## Pacotes

In [3]:
using JuMP, HiGHS, LinearAlgebra

## Modelo Cru

In [4]:
model = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

## Variáveis de Decisão

In [5]:
# Lasanhas em unidades por mês.
@variable(model, l[T] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
And data, a 4-element Vector{VariableRef}:
 l[1]
 l[2]
 l[3]
 l[4]

Pergunta 2: Vocês sentem falta de alguma decisão? Qual?

## Variáveis Auxiliares

In [6]:
# Estoque de lasanhas no início de cada mês.
@variable(model, e[T] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
And data, a 4-element Vector{VariableRef}:
 e[1]
 e[2]
 e[3]
 e[4]

Pergunta 3: Por que essa última variável é só auxiliar, ou seja, não é imprensindível ao modelo?

## Restrições

In [7]:
# Fixando a decisão do passado.
@constraint(model, FP, e[1] == ei)
# Balanço do estoque.
@constraint(model, BE[t = T[1:end-1]], e[t+1] == e[t] + l[t] - d[t]);
# Atendimento à demanda (necessária apenas para t=4, redundante para 1 <= t <= 3).
@constraint(model, AD[t = T], l[t] + e[t] >= d[t]);

Pergunta 4: por que a restrição de "Atendimento à demanda" é desnecessária, nesse caso?

## Função Objetivo

In [8]:
@objective(model, Min, dot(cp,l) + dot(ce,e[2:end]));

## Modelo Completo

In [9]:
print(model)

## Resolvendo o Modelo

In [10]:
optimize!(model)

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 8 rows; 8 cols; 18 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [1e+00, 1e+01]
  Bound   [0e+00, 0e+00]
  RHS     [5e+01, 4e+02]
Presolving model
5 rows, 6 cols, 12 nonzeros  0s
3 rows, 4 cols, 7 nonzeros  0s
1 rows, 2 cols, 2 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-8); columns 0(-8); nonzeros 0(-18) - Reduced to empty
Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Objective value     :  7.2425000000e+03
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


## Solução

In [11]:
println("l = ", value(l))
println("e = ", value(e))
println("Objective = ", objective_value(model))

l = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
And data, a 4-element Vector{Float64}:
 650.0
   0.0
   0.0
 250.0
e = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
And data, a 4-element Vector{Float64}:
  50.0
 500.0
 150.0
   0.0
Objective = 7242.5


# Complexidades adicionais

## Capacidade de estoque

Após receber o planejamento de produção ótimo acima, o dono da Pastesian, Ricardo das Massas, notou que armazenar 500 unidades de lasanha não é viável pois a empresa não possui tantos refrigeradores.

Ele percebeu que é necessário restringir o modelo para que seja respeitada uma capacidade de estoque de no máximo 200 lasanhas de um mês para o próximo.

### Restrições Adicionais

In [12]:
# Capacidade de estoque
@constraint(model, CE[t = T], e[t] <= 200);

In [13]:
print(model)

### Reotimizando

In [14]:
optimize!(model)

LP has 12 rows; 8 cols; 22 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [1e+00, 1e+01]
  Bound   [0e+00, 0e+00]
  RHS     [5e+01, 4e+02]
Solving LP with useful basis so presolve not used
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     7.2425673190e+03 Pr: 1(300) 0s
          2     7.3100000000e+03 Pr: 0(0) 0s

Model status        : Optimal
Simplex   iterations: 2
Objective value     :  7.3100000000e+03
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


### Solução Atualizada

In [15]:
println("l = ", value(l))
println("e = ", value(e))
println("Objective = ", objective_value(model))

l = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
And data, a 4-element Vector{Float64}:
 350.0
 150.0
 150.0
 250.0
e = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
And data, a 4-element Vector{Float64}:
  50.0
 200.0
  -0.0
   0.0
Objective = 7310.0


## Incertezas

Ricardo lembrou que a demanda para os próximos 4 meses não é conhecida precisamente, mas uma previsão. Em temporadas anteriores, houve uma variação significativa na demanda do mês mais distante, o que prejudicou o planejamento.

Neste ano, Ricardo pretende se planejar para essa incerteza na demanda. Agora, ele disponibilizou três previsões de demanda para o mês 4: 220, 250, e 300 lasanhas. Como poderíamos planejar a produção de lasanhas nesse cenário?

### Novos parâmetros

Vamos modificar o parâmetro de demanda.

In [16]:
# Lista de cenários possíveis.
C = [1, 2, 3];
# Demanda (unidades) no último mês, a depender do cenário.
dend = [220, 250, 300]
# Penalidade de produção a menos.
# Valor de revenda maior que 3 vezes o valor de produção.
# pp = 22;
# Valor de revenda maior que 2 vezes o valor de produção.
#pp = 11;
# Valor de revenda igual a 1.5 vezes o valor de produção.
pp = 5.045;

### Novo Modelo

In [17]:
model2 = Model(HiGHS.Optimizer);

### Novas Variáveis de Decisão

In [18]:
# Lasanhas produzidas (unidades) por mês (exceto o último).
@variable(model2, l[T[1:end-1]] >= 0);
# Estoque de lasanhas em cada mês.
@variable(model2, e[T] >= 0);
# Lasanhas produzidas (unidades) no último mês, em cada cenário.
@variable(model2, lc[C] >= 0)
# Diferença positiva entre a demanda e o produzido (ou seja, demanda não atendida no último mês), para cada cenário.
@variable(model2, dp[C] >= 0);

### Novas Restrições

In [19]:
# Fixando a decisão do passado.
@constraint(model2, FP, e[1] == ei)
# Balanço do estoque.
@constraint(model2, BE[t = T[1:end-1]], e[t+1] == e[t] + l[t] - d[t])
# Atendimento à demanda (desnecessária para períodos 1 <= t <= 3, incorreta para t = 4 devido à modelagem/restrição com dp[c] abaixo).
# @constraint(model2, AD[t = T[1:end-1]], l[t] + e[t] >= d[t])
# Relação para cada cenário entre produzido e demanda.
@constraint(model2, ADC[c = C], dp[c] >= dend[c] - (lc[c] + e[T[end]]));

In [20]:
# As decisões devem ser uma só, independentemente do cenário.

# Restrição de decisões equivalentes para cada cenário ("no antecipative constraint").
@constraint(model2, DE[c = C], sum(lc) == length(C) * lc[c] )

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [1, 2, 3]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 DE[1] : -2 lc[1] + lc[2] + lc[3] = 0
 DE[2] : lc[1] - 2 lc[2] + lc[3] = 0
 DE[3] : lc[1] + lc[2] - 2 lc[3] = 0

### Nova Função Objetivo

In [21]:
@objective(model2, Min, dot(cp[1:end-1],l) +  cp[end] * lc[1] + dot(ce,e[2:end]) + pp*sum(dp)/length(C));

### Modelo Completo

In [22]:
print(model2)

### Resolvendo o Modelo

In [23]:
optimize!(model2)

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 10 rows; 13 cols; 28 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 2e+00]
  Cost    [1e+00, 1e+01]
  Bound   [0e+00, 0e+00]
  RHS     [5e+01, 4e+02]
Presolving model
8 rows, 11 cols, 24 nonzeros  0s
5 rows, 5 cols, 11 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-10); columns 0(-13); nonzeros 0(-28) - Reduced to empty
Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Objective value     :  5.8123833333e+03
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.00


### Solução

In [24]:
println("l = ", value(l))
println("e = ", value(e))
println("lc =", value(lc))
println("dp = ", value(dp))
println("Objective = ", objective_value(model2))

l = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3]
And data, a 3-element Vector{Float64}:
 650.0
   0.0
   0.0
e = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3, 4]
And data, a 4-element Vector{Float64}:
  50.0
 500.0
 150.0
   0.0
lc =1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3]
And data, a 3-element Vector{Float64}:
 -0.0
 -0.0
  0.0
dp = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [1, 2, 3]
And data, a 3-element Vector{Float64}:
 220.0
 250.0
 300.0
Objective = 5812.383333333333


Pergunta 5: O que muda no modelo acima se os cenários não forem equiprováveis?

# Desafio

A empresa decidiu lançar um novo sabor de lasanha para esta temporada. Enquanto a demanda do sabor tradicional é conhecida a partir de dados históricos e bastante previsível, a demanda para o novo sabor é incerta, especialmente no mês 4 que está mais distante no planejamento.

Estima-se que a demanda do novo sabor seja de 30, 70, e 140 para os meses 1, 2, e 3. Para o mês 4, após a popularidade aumentar, a demanda pode assumir os seguintes valores com as respectivas probabilidades esperadas:
| Demanda (unidades) | Probabilidade |
|----------|---------------|
| 200 | 0,3 |
| 240 | 0,5 |
| 300 | 0,2 |

Além disso, Ricardo pretende expandir seu estoque reinvestindo dinheiro ao longo da temporada. Dessa forma, a capacidade de estoque varia da seguinte forma:
| Mês | Capacidade de estoque (unidades) |
|----------|---------------|
| 1 | 200 |
| 2 | 220 |
| 3 | 230 |
| 4 | 250 |

Considerando as novas complexidades trazidas por Ricardo das Massas frente ao crescimento do seu negócio, vamos às **etapas do desafio**:

1. Construa, implemente e resolva um **modelo determinístico** (sem incerteza na demanda do mês 4) que utilize a demanda **média** esperada para o mês 4.
2. Construa, implemente e resolva um **modelo estocástico**, que leve em consideração os três cenários possíveis para a demanda no mês 4 e suas respectivas probabilidades esperadas.
3. Compare as soluções obtidas por cada modelo acima destacando os valores ótimos das variáveis e função objetivo.


In [ ]:
# adicionar restrição de capacidade de estoque (que n tem no problema anterior)
# considerar a lasanha antiga e nova
# custos da nova lasanha é o mesmo que o da antiga
# considerar estoque inicial 0
# Considerar demandas separadas
# Capacidade de produção, e o excendente pagar o custo de produção de R$0,30 (verificar o site)

## Parâmetros

In [36]:
# Período
T = 1:4;

# Restrições para a nova lasanha

# Estoque de cada mês
E = [200, 220, 230, 250]; # estoque maximo para as duas lasanhas ([estoque lasanha antiga no mes 1] + [estoque lasanha nova no mes 1] = 200 ...)
# Custo de estoque

# Capacidade de produção infinito ?



# Custo de produção por unidade de lasanha


# Demanda dos meses 1 a 3
D_n = [30, 70, 140];
# Demanda (em unidades) para mês 4
D4_n = [200, 240, 300];
# Probabilidade de cada demanda ocorrer no mês 4
P4_n = [0.3, 0.5, 0.2];


## Modelo determinístico

In [59]:
d4 = D4' * P4
D_deterministico = vcat(D, [d4])

4-element Vector{Float64}:
  30.0
  70.0
 140.0
 240.0

In [56]:
model_deterministico = Model(HiGHS.Optimizer);

**Variáveis**

In [57]:
# Lasanhas produzidas em unidades por mês
@variable(model_deterministico, l[T] >= 0);
# Estoque de lasanhas no início de cada mês
@variable(model_deterministico, e[T] >= 0);

**Restrições**

In [58]:
# Capacidade de estoque
@constraint(model_deterministico, CE[t = T], e[t] <= E[t]);


## Modelo estocástico

In [48]:
model_estocastico = Model(HiGHS.Optimizer);

**Variáveis**

In [49]:
# Lasanhas produzidas em unidades por mês
@variable(model_estocastico, l[T] >= 0);
# Estoque de lasanhas no início de cada mês
@variable(model_estocastico, e[T] >= 0);

**Restrições**

In [ ]:
# Capacidade de estoque
@constraint(model_estocastico, CE[t = T], e[t] <= E[t]);

# Referências

- Mais use-cases contextualizados (como o Pastesian): https://www.mipwise.com/use-cases
- Puzzles (menos contexto): https://www.mipwise.com/puzzles